## Assignment 3: Clustering Models

In this assignment, you will implement the K-means method and apply it to image compression. Then you will use the Gaussian Mixture Models from the scikit-learn library and select the number of clusters K using the Akaike Information Criterion (AIC) and the Bayesian Information Criterion (BIC).

#### Part 1) Implement the K-means algorithm

In [2]:
import numpy as np
import pandas as pd
import random
import math
from collections import Counter

'''
Process of K-means:
    1. Select the desired amount of clusters, K.
    2. Randomly select K distinct data points; these are the initial clusters.
    3. Measure the distance between the first data point and all the K initial clusters. For 2D, euclidian distance.
    4. Assign the first point to its nearest cluster. Do the same thing for all the remaining points.
    5. Calculate the mean of each cluster.
    6. Then we repeat the process, but this time with the mean values as the clusters (cluster points).
    The result of the clusters produced by this method is measured by the variation within the clusters.
    The model cannot "see" the clusters, it can only determine weather the clusters are good or not with 
    measuring the variation within and then sum the variation. 
    Since the initial points are chosen at random, it is good practice to repeat the above method 
    multiple times, with different randomised initial point, and for each result
    measure the variation within each cluster and determine which one produced the best clusters 
    by comparing them with each other.  

    Pick K by finding the "elbow" in an "eblow-plot". The "elbow-plot" is a plot of the reduction
    in variance per value of K.

'''

# Sample n random distinct points from data x 
def sample_random_points(x, n: int):
    """ random_points = {} # {index: point}
    for i in range(n):
        rand = random.randint(0,len(x)-1)
        while x[rand] in random_points.values():
            rand = random.randint(0,len(x)-1)
        random_points.update({i: x[rand]})
    return random_points """
    random_points = {}
    random_sample = random.sample(list(x), n)
    for i in range(n):
        random_points.update({i: random_sample[i]})

    return random_points

# Calculates the euclidean distance between two points p and q in 2D space
def euclid_distance(p, q):
    p1, p2 = p
    q1, q2 = q
    distance = math.sqrt((q1-p1)**2 + (q2-p2)**2)
    return distance

# Calculates the euclidean distance between a point and the given centroids
# Returns the index of the centroids which is closest to the point
def calc_distance_to_centroids(p, centroids):
    closest_centroid_index = -1
    min_distance = float('inf') # Initialised to max float value
    for i in range(len(centroids)):
        distance = euclid_distance(p, centroids[i])
        if distance <= min_distance:
            min_distance = distance
            closest_centroid_index = i
    return closest_centroid_index

def compute_mean_values(x, indices):
    groups = {} # {label: [data points]}
    # For every data point add it to the corresponding centroid index
    for i in range(len(x)):
        if indices[i] not in groups:
            groups.update({indices[i]: [x[i]]})
        else:
            groups[indices[i]].append(x[i])
    # Calculate the mean value for every data point for each index
    mean_values = {}
    for label in groups.keys():
        values = groups[label]
        mean = sum(values) / len(values) 
        mean_values.update({label: mean})
    return mean_values





def kmeans(x, K: int, n_init: int):   
    # x: input data     
    # K: number of centroids
    # n_init: the number of initial guesses for the centroids
    # centroids: contains the centers of the clusters
    # labels: contains the cluster index for each data point 

    centroids = []
    labels = [] 

    for _ in range(n_init):
        tmp_centroids = sample_random_points(x, K)

        not_converged = True
        # Loop until convergence
        while not_converged:
            tmp_labels = []
            print('--------------- New iteration')
            for xn in x:
                cluster_index = calc_distance_to_centroids(xn, list(tmp_centroids.values()))
                tmp_labels.append(cluster_index)
            updated_centroids = compute_mean_values(x, tmp_labels) 
            # Disclaimer, the centroids might not be placed at the same index <- changed to dict with indices, so should be ok

            # Check if the centroids have converged
            check = True
            while check:
                #print(tmp_centroids)
                for i in range(len(tmp_centroids.values())):
                    l = np.array(updated_centroids[i]) == np.array(tmp_centroids[i])
                    #print(l)
                    if not l.all():
                        check = False 
                        # No convergence, updated the centroids
                        tmp_centroids = updated_centroids
                    else:
                        print(tmp_centroids)
                        centroids = tmp_centroids
                        labels = tmp_labels
                        print('Is this reached?')
                        not_converged = False
                        break
            # If the check is true, the centroids have converged
            '''if check:
                centroids = tmp_centroids
                labels = tmp_labels
                print('Is this reached?')
                break
            '''

    # Evaluate the scores somehow and then return the best ones

    return centroids, labels

print(sample_random_points([1,2,3,4,5,6,7,8,9], 3))
print(euclid_distance((1,1), (2,2)))
d = {}
d.update({'hej':1})
d['nej'] = 2
print(d)

{0: 3, 1: 2, 2: 9}
1.4142135623730951
{'hej': 1, 'nej': 2}


In [3]:
# Testing the algorithm
from sklearn.datasets import make_blobs
features, true_labels = make_blobs(
    n_samples = 200,
    centers = 3,
    cluster_std= 3,
    random_state= 3
)
#print(features)
print([1,2] == [1,2])
r = np.array([1,3,4]) == np.array([1,3,5])
print(r)
print(r.all())

centroids, labels = kmeans(features, 3, 1)
print(centroids)
print('-------')
print('labels:', labels)
print('true labels:', true_labels)
trues = 0
falses = 0
for label, true_label in zip(labels, true_labels):
    if label == true_label:
        trues += 1
    else:
        falses += 1
print(trues)
print(falses)

True
[ True  True False]
False
--------------- New iteration
{1: array([-4.84597226, -0.92118063]), 0: array([2.52641778, 4.04172888]), 2: array([ 5.93895307, 10.21234286])}
Is this reached?
{1: array([-4.84597226, -0.92118063]), 0: array([2.52641778, 4.04172888]), 2: array([ 5.93895307, 10.21234286])}
-------
labels: [1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 2, 0, 0, 1, 0, 0, 0, 1, 0, 2, 0, 2, 0, 0, 1, 0, 2, 0, 0, 0, 2, 1, 1, 2, 1, 1, 0, 2, 1, 0, 1, 0, 0, 0, 2, 1, 2, 0, 0, 0, 2, 2, 1, 0, 0, 0, 1, 0, 1, 0, 2, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 2, 1, 0, 0, 2, 0, 2, 2, 2, 2, 0, 0, 1, 0, 0, 0, 2, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 1, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 2, 0, 0, 0, 0, 0, 0, 2, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 2, 0, 0, 0, 1, 0, 0, 0, 0, 0, 2, 1, 0, 2, 2, 1, 0, 0, 0, 1, 2, 2, 1, 0, 0, 1, 1, 1, 0, 0, 2, 2, 0, 2, 0, 0, 2, 0, 0, 2, 0, 0, 0, 0, 1, 0]
true labels: [1 0 2 1 2 1 0 0 1 1 1 0 0 2 0 0 1 2 2 1 0 2 2 2 0 1 1 0 0 0 0 0 2

In [16]:
# SSE and Siloutte scores

# Summation of all the distances from each point to its given cluster (centroid)
def sse(x, K, labels, centroids):
    sse_result = [0 for i in range(K)]
    for point, label in zip(x, labels):
        sse_result[label] += np.square(euclid_distance(point, centroids[label]))
    return sse_result

print(sse(features, 3, labels, centroids))

# Computes the distance from a given data point to all other data points
# within the same cluster
# TODO: Check correctness!
def within_cluster_distance(x_labels, cluster, data_point):
    #x_labels = zip(x, labels)
    clusters = Counter(x_labels)
    cardinality = cluster[cluster]
    acc_distance = 0
    a_i = 0

    for point, label in x_labels:
        if point != data_point and label == cluster:
            acc_distance += euclid_distance(data_point, point)
    
    a_i = (1 / (cardinality - 1)) * acc_distance
    return a_i

# Computes the distance from a given data point to all other data points
# outside of its cluster
# TODO: Check correctness!
def between_cluster_distance(x_labels, K, cluster, data_point):
    #x_labels = zip(x, labels) 
    cluster_count = Counter(x_labels)
    other_clusters = [0 for i in range(K-1)]

    for point, label in x_labels:
        if label != cluster:
            other_clusters[label] += euclid_distance(data_point, point)
    
    for c_k in other_clusters:
        other_clusters[c_k] = (1/cluster_count[c_k]) * other_clusters[c_k]
    
    return min(other_clusters)




# A mixture of distance between points within the cluster and distance between points from
# the other clusters, i.e. outside of the given points cluster.
def siloutte_score(x, K, labels):
    x_labels = zip(x, labels)
    score = 0
    for point, label in x:
        a_i = within_cluster_distance(x_labels, label, point) 
        b_i = between_cluster_distance(x_labels, K, label, point)
        score += ((b_i - a_i)/np.max([b_i, a_i]))
    return score/float(len(x))



#list(zip(features, labels))


[3632.4177681873507, 497.867693158382, 830.8086686750097]


* **If you run this algorithm multiple times, do you get the same result every time? If so, why; if not, how do you determine which result is the best one?**
    
    The result isn't the same, since the algorithm involves randomness. 
    The K initial points, centroids for the K clusters, are chosen at random. Thus, each iteration of the algorithm
    could result in different results, and more often than not do.
    

* **How do you choose what K value to use?**

    Elbow-graph
* **Implement both SSE and the Silhouette score; use them to address these questions.**

    See code cell above.


* ***Discuss the similarities and differences between 1) K-means, 2) Gaussian Mixture Models (GMM) and 3) the Gaussian naive Bayes classifier in terms of their assumptions and parameter estimation methods. In particular, write down the implications of each assumption (e.g., x and y are assumed to be independent, and therefore, we have ....).  You can choose to write your answers as Python code, pseudo code or mathematical equations.**

#### Part 2) Apply the K-means algorithm to compress images

#### Part 3) Use AIC and BIC to choose K for Gaussian Mixture Models 

In [ ]:
from sklearn.datasets import load_breast_cancer
data = load_breast_cancer().data 